# A real world workchain example: electronic band structure

*Import statements - make sure to execute the cell below this one (it may be hidden)*

In [ ]:
from datetime import datetime, timedelta
from aiida import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()

from aiida.orm import WorkflowFactory, Code
from aiida.orm.data.base import Str
from aiida.tools.dbimporters import DbImporterFactory
from aiida.work.run import run

PwBandStructureWorkChain = WorkflowFactory('quantumespresso.pw.band_structure')

### Calculating the electronic band structure with an AiiDA workchain
This tutorial will show how useful a workchain can be in performing a well defined task, such as computing and visualizing the electronic band structure for a simple crystal structure. The goal of this tutorial is not to show you the intricacies of the actual workchain itself, but rather to serve as an example that workchains can simplify standard workflows in computational materials science. The workchain that we will use here will employ Quantum Espresso's pw.x code to calculate the charge densities for several crystal structures and compute a band structure from those. Many choices that normally face the researcher before being able to perform this calculation, such as the selection of suitable pseudo potentials, energy cutoff values, k-point grids and k-point paths along high symmetry points, are now performed automatically by the workchain. All that remains for the user to do is to simply define a structure, pass it to the workchain and sit back!

*Disclaimer: the part of the workchain that will try to self-consistently converge the charge density, written specifically for this demonstration, is a stripped down version and as such has only limited capabilities in the handling of completed pw.x calculations that did not successfully converge. However, the workchain is written in such a way, providing function stubs where this error handling code could be written, that you can easily take this as a template and implement this functionality yourself.*

Below, we have already imported four different crystal structures as an example. Note that the time to run the full workchain for each of these can be very different. The estimated run time is noted in a comment in the corresponding calculation cell. It is not necessary to run all of them.

In [ ]:
# Loading the COD importer so we can directly import structure from COD id's
CodImporter = DbImporterFactory('cod')
importer = CodImporter()

# Make sure here to define the correct codename that corresponds to the pw.x code installed on your machine of choice
codename = 'qe-pw-6.2.1@localhost'
code = Code.get_from_string(codename)

#### Importing example crystal structures from COD to AiiDA structure objects

In [ ]:
# Al COD ID(9008460)
structure_Al = importer.query(id='9008460')[0].get_aiida_structure()
structure_Al.get_formula()

In [ ]:
# GaAs COD ID(9008845)
structure_GaAs = importer.query(id='9008845')[0].get_aiida_structure()
structure_GaAs.get_formula()

In [ ]:
# CaF2 COD ID(1000043)
structure_CaF2 = importer.query(id='1000043')[0].get_aiida_structure()
structure_CaF2.get_formula()

In [ ]:
# h-BN COD ID(9008997)
structure_hBN = importer.query(id='9008997')[0].get_aiida_structure()
structure_hBN.get_formula()

#### Now we run the bandstructure workchain for the selected structures

The bandstructure workchain follows the following protocol:
* Determine the primitive cell of the input structure
* Run a vc-relax to relax the structure
* Refine the symmetry of the relaxed structure to ensure the primitive cell is used and run a self-consistent field calculation on it
* Run a non self-consistent field band structure calculation along a path of high symmetry k-points determined by [seekpath](http://materialscloud.org/tools/seekpath)

Numerical parameters for the default 'standard' protocol are determined as follows:
* Suitable pseudo potentials and energy cutoffs are automatically chosen from the [SSSP library](http://materialscloud.org/sssp) (efficiency version)
* K-point mesh is selected to have a minimum k-point density of 0.2 Å<sup>-1</sup>
* A Marzari-Vanderbilt smearing of 0.02 Ry is used for the electronic occupations

In [ ]:
# This will take approximately 3 minutes on the tutorial AWS
results_Al = run(
    PwBandStructureWorkChain,
    code=code,
    pseudo_family=Str('SSSP'),
    structure=structure_Al,
)

In [ ]:
fermi_energy = results_Al['scf_parameters'].dict.fermi_energy
results_Al['bandstructure'].show_mpl(y_origin=fermi_energy, plot_zero_axis=True)

print """Final crystal symmetry: {spacegroup_international} (number {spacegroup_number})
Extended Bravais lattice symbol: {bravais_lattice_extended}
The system has inversion symmetry: {has_inversion_symmetry}""".format(
    **results_Al['final_seekpath_parameters'].get_dict())

In [ ]:
# This will take approximately 5 minutes on the tutorial AWS
results_GaAs = run(
    PwBandStructureWorkChain,
    code=code,
    pseudo_family=Str('SSSP'),
    structure=structure_GaAs,
)

In [ ]:
fermi_energy = results_GaAs['scf_parameters'].dict.fermi_energy
results_GaAs['bandstructure'].show_mpl(y_origin=fermi_energy, plot_zero_axis=True)

print """Final crystal symmetry: {spacegroup_international} (number {spacegroup_number})
Extended Bravais lattice symbol: {bravais_lattice_extended}
The system has inversion symmetry: {has_inversion_symmetry}""".format(
    **results_GaAs['final_seekpath_parameters'].get_dict())

In [ ]:
# This will take approximately 9 minutes on the tutorial AWS
results_CaF2 = run(
    PwBandStructureWorkChain,
    code=code,
    pseudo_family=Str('SSSP'),
    structure=structure_CaF2,
)

In [ ]:
fermi_energy = results_CaF2['scf_parameters'].dict.fermi_energy
results_CaF2['bandstructure'].show_mpl(y_origin=fermi_energy, plot_zero_axis=True)

print """Final crystal symmetry: {spacegroup_international} (number {spacegroup_number})
Extended Bravais lattice symbol: {bravais_lattice_extended}
The system has inversion symmetry: {has_inversion_symmetry}""".format(
    **results_CaF2['final_seekpath_parameters'].get_dict())

In [ ]:
# This will take approximately 28 minutes on the tutorial AWS
results_hBN = run(
    PwBandStructureWorkChain,
    code=code,
    pseudo_family=Str('SSSP'),
    structure=structure_hBN,
)

In [ ]:
fermi_energy = results_hBN['scf_parameters'].dict.fermi_energy
results_hBN['bandstructure'].show_mpl(y_origin=fermi_energy, plot_zero_axis=True)

print """Final crystal symmetry: {spacegroup_international} (number {spacegroup_number})
Extended Bravais lattice symbol: {bravais_lattice_extended}
The system has inversion symmetry: {has_inversion_symmetry}""".format(
    **results_hBN['final_seekpath_parameters'].get_dict())